### Download the necessary modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

### Import the data in excel file and format them

In [237]:
df_raw =  pd.read_excel('séries_AirLiquide.xlsx')
df_raw.head(10)

,Unnamed: 0,Div AirLiquide (lissé),Prix Airliquide,Indice CAC40
0,1987M03,0.305684,11.1670,NaN
1,1987M04,0.306773,11.8679,NaN
2,1987M05,0.307862,11.3422,NaN
3,1987M06,0.308952,11.0714,NaN
4,1987M07,0.310043,10.9918,NaN
5,1987M08,0.311138,11.1511,1483.02
6,1987M09,0.312236,11.0873,1524.00
7,1987M10,0.313339,10.8962,1456.88
8,1987M11,0.314448,8.5226,1116.47
9,1987M12,0.315563,7.9969,1026.98


In [238]:
df = df_raw
df.head(10)

,Unnamed: 0,Div AirLiquide (lissé),Prix Airliquide,Indice CAC40
0,1987M03,0.305684,11.1670,NaN
1,1987M04,0.306773,11.8679,NaN
2,1987M05,0.307862,11.3422,NaN
3,1987M06,0.308952,11.0714,NaN
4,1987M07,0.310043,10.9918,NaN
5,1987M08,0.311138,11.1511,1483.02
6,1987M09,0.312236,11.0873,1524.00
7,1987M10,0.313339,10.8962,1456.88
8,1987M11,0.314448,8.5226,1116.47
9,1987M12,0.315563,7.9969,1026.98


In [240]:
df['year']=[x.split("M")[0] for x in df['Unnamed: 0']]
df['month']=[x.split("M")[1] for x in df['Unnamed: 0']]
df.head(10)

,Unnamed: 0,Div AirLiquide (lissé),Prix Airliquide,Indice CAC40,year,month
0,1987M03,0.305684,11.1670,NaN,1987,03
1,1987M04,0.306773,11.8679,NaN,1987,04
2,1987M05,0.307862,11.3422,NaN,1987,05
3,1987M06,0.308952,11.0714,NaN,1987,06
4,1987M07,0.310043,10.9918,NaN,1987,07
5,1987M08,0.311138,11.1511,1483.02,1987,08
6,1987M09,0.312236,11.0873,1524.00,1987,09
7,1987M10,0.313339,10.8962,1456.88,1987,10
8,1987M11,0.314448,8.5226,1116.47,1987,11
9,1987M12,0.315563,7.9969,1026.98,1987,12


In [242]:
df['log_return_price'] = np.log(df['Prix Airliquide']/df['Prix Airliquide'].shift(1)) # p(t)/p(t-1)
df['log_return_CAC40'] = np.log(df['Indice CAC40']/df['Indice CAC40'].shift(1)) # nbrepoints(t)/nbrepoints(t-1)
df

,Unnamed: 0,Div AirLiquide (lissé),Prix Airliquide,Indice CAC40,year,month,log_return_price,log_return_CAC40
0,1987M03,0.305684,11.1670,NaN,1987,03,NaN,NaN
1,1987M04,0.306773,11.8679,NaN,1987,04,0.060874,NaN
2,1987M05,0.307862,11.3422,NaN,1987,05,-0.045307,NaN
3,1987M06,0.308952,11.0714,NaN,1987,06,-0.024165,NaN
4,1987M07,0.310043,10.9918,NaN,1987,07,-0.007216,NaN
...,...,...,...,...,...,...,...,...
392,2019M11,2.465649,118.4500,5761.89,2019,11,0.041088,0.060659
393,2019M12,2.474304,119.6000,5786.74,2019,12,0.009662,0.004304
394,2020M01,2.482985,126.7000,6041.50,2020,01,0.057669,0.043083
395,2020M02,2.491688,131.9000,5832.51,2020,02,0.040222,-0.035205


In [243]:
#COVAR(O3:O394;$Y$3:$Y$394)/VARP($Y$3:$Y$394)
year = '2020'
ret_price_year = df['log_return_price'][df['year'] == year]
ret_CAC_40_year = df['log_return_CAC40'][df['year'] == year]

x = ret_price_year.values
y = ret_CAC_40_year.values

matrix_xy = np.stack((x, y), axis=0)
print(ret_price_year)
print(ret_CAC_40_year)
matrix_xy

394    0.057669
395    0.040222
396   -0.053330
Name: log_return_price, dtype: float64
394    0.043083
395   -0.035205
396   -0.089436
Name: log_return_CAC40, dtype: float64


array([[ 0.05766925,  0.04022197, -0.0533304 ],
       [ 0.04308323, -0.03520489, -0.08943601]])

In [244]:
beta = []
years = df['year'].unique()
for year in years :
    ret_price_year = df['log_return_price'][df['year'] == year]
    ret_CAC_40_year = df['log_return_CAC40'][df['year'] == year]

    x = ret_price_year.values
    y = ret_CAC_40_year.values

    matrix_xy = np.stack((x, y), axis=0)
    
    var_y = np.var(y)
    cov_matrix = np.cov(matrix_xy)
    cov_xy = cov_matrix[1,0]
    beta_xy = cov_xy/var_y
    
    beta.append(beta_xy)
    
print(len(beta))
print(len(years))

34
34


In [245]:
dividend_yearly = df.groupby('year')['Div AirLiquide (lissé)'].mean().round(3)
price_yearly = df.groupby('year')['Prix Airliquide'].mean().round(3)
indiceCAC40_yearly = df.groupby('year')['Indice CAC40'].mean().round(3)
return_IndiceCAC40_yearly = df.groupby('year')['log_return_CAC40'].mean().round(3)

In [246]:
df_yearly = pd.concat([dividend_yearly, price_yearly, indiceCAC40_yearly, return_IndiceCAC40_yearly], axis=1)
df_yearly['beta'] = beta
df_yearly.head(10)

,Div AirLiquide (lissé),Prix Airliquide,Indice CAC40,log_return_CAC40,beta
year,,,,,
1987,0.311,10.609,1321.470,-0.092,NaN
1988,0.323,8.497,1229.658,0.030,1.170711
1989,0.336,10.926,1750.667,0.022,1.544345
1990,0.350,12.387,1859.954,-0.013,0.898183
1991,0.371,12.834,1764.264,0.003,0.803905
1992,0.398,14.827,1846.913,0.004,0.500614
1993,0.425,16.328,2012.700,0.016,0.904070
1994,0.448,17.275,2065.525,-0.007,0.845330
1995,0.469,18.181,1869.042,-0.007,0.381206


In [247]:
df_yearly['Dividend'] = df_yearly['Div AirLiquide (lissé)']
df_yearly['Price'] = df_yearly['Prix Airliquide']
df_yearly['CAC40_basis_points'] = df_yearly['Indice CAC40']
df_yearly['return_CAC40'] = df_yearly['log_return_CAC40']
df_yearly.drop(columns = ['Div AirLiquide (lissé)','Prix Airliquide','Indice CAC40','log_return_CAC40'],inplace=True)
df_yearly

,beta,Dividend,Price,CAC40_basis_points,return_CAC40
year,,,,,
1987,NaN,0.311,10.609,1321.470,-0.092
1988,1.170711,0.323,8.497,1229.658,0.030
1989,1.544345,0.336,10.926,1750.667,0.022
1990,0.898183,0.350,12.387,1859.954,-0.013
1991,0.803905,0.371,12.834,1764.264,0.003
1992,0.500614,0.398,14.827,1846.913,0.004
1993,0.904070,0.425,16.328,2012.700,0.016
1994,0.845330,0.448,17.275,2065.525,-0.007
1995,0.381206,0.469,18.181,1869.042,-0.007


In [248]:
df_yearly['Dividend_Rate'] = df_yearly['Dividend'].pct_change(1)
df_yearly

,beta,Dividend,Price,CAC40_basis_points,return_CAC40,Dividend_Rate
year,,,,,,
1987,NaN,0.311,10.609,1321.470,-0.092,NaN
1988,1.170711,0.323,8.497,1229.658,0.030,0.038585
1989,1.544345,0.336,10.926,1750.667,0.022,0.040248
1990,0.898183,0.350,12.387,1859.954,-0.013,0.041667
1991,0.803905,0.371,12.834,1764.264,0.003,0.060000
1992,0.500614,0.398,14.827,1846.913,0.004,0.072776
1993,0.904070,0.425,16.328,2012.700,0.016,0.067839
1994,0.845330,0.448,17.275,2065.525,-0.007,0.054118
1995,0.381206,0.469,18.181,1869.042,-0.007,0.046875


In [249]:
# Year 1987 should be removed because there is not enough data for the CAC 40 index
df_yearly.drop(['1987'], axis=0, inplace=True)
df_yearly 

,beta,Dividend,Price,CAC40_basis_points,return_CAC40,Dividend_Rate
year,,,,,,
1988,1.170711,0.323,8.497,1229.658,0.030,0.038585
1989,1.544345,0.336,10.926,1750.667,0.022,0.040248
1990,0.898183,0.350,12.387,1859.954,-0.013,0.041667
1991,0.803905,0.371,12.834,1764.264,0.003,0.060000
1992,0.500614,0.398,14.827,1846.913,0.004,0.072776
1993,0.904070,0.425,16.328,2012.700,0.016,0.067839
1994,0.845330,0.448,17.275,2065.525,-0.007,0.054118
1995,0.381206,0.469,18.181,1869.042,-0.007,0.046875
1996,0.786398,0.492,20.877,2082.438,0.020,0.049041


In [250]:
french_treasuries_bonds =  pd.read_excel('tx_dintéret_OAT.xlsx',sheet_name = 'Final', header = None)
french_treasuries_bonds.columns = ['year','french_bonds_rate']
french_treasuries_bonds['year'] = french_treasuries_bonds['year'].astype(str)
french_treasuries_bonds.set_index('year', inplace = True)
french_treasuries_bonds.head(10)

,french_bonds_rate
year,
1999,3.410000
2000,0.700000
2001,3.280000
2002,3.310000
2003,2.450000
2004,2.370000
2005,4.210000
2006,2.910000
2007,1.875000


### Now we need to cross the intervals of time with data available from both dataframes. in our case, the data are fine between 1999 and 2020

In [251]:
french_treasuries_bonds.drop(['2021', '2022'], axis = 0, inplace = True)
french_treasuries_bonds

,french_bonds_rate
year,
1999,3.410000
2000,0.700000
2001,3.280000
2002,3.310000
2003,2.450000
2004,2.370000
2005,4.210000
2006,2.910000
2007,1.875000


In [252]:
df_yearly.iloc[:11,:].index

Index(['1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998'],
      dtype='object', name='year')

In [253]:
df_yearly.drop(df_yearly.iloc[:11,:].index, axis = 0, inplace = True)
df_yearly

,beta,Dividend,Price,CAC40_basis_points,return_CAC40,Dividend_Rate
year,,,,,,
1999,0.750548,0.655,27.822,4464.609,0.032,0.129310
2000,0.150562,0.752,29.066,6353.173,0.008,0.148092
2001,-0.121145,0.870,32.785,5085.517,-0.024,0.156915
2002,0.421490,0.979,33.140,3866.780,-0.025,0.125287
2003,0.622865,1.038,30.311,3115.573,0.004,0.060266
2004,0.807549,1.027,33.690,3699.040,0.008,-0.010597
2005,0.806305,0.988,37.058,4247.107,0.017,-0.037975
2006,1.038258,1.009,44.525,5095.468,0.010,0.021255
2007,0.505131,1.103,52.345,5752.749,0.006,0.093162


In [254]:
df_yearly_final = pd.concat([df_yearly,french_treasuries_bonds], axis = 1)

df_yearly_final

,beta,Dividend,Price,CAC40_basis_points,return_CAC40,Dividend_Rate,french_bonds_rate
year,,,,,,,
1999,0.750548,0.655,27.822,4464.609,0.032,0.129310,3.410000
2000,0.150562,0.752,29.066,6353.173,0.008,0.148092,0.700000
2001,-0.121145,0.870,32.785,5085.517,-0.024,0.156915,3.280000
2002,0.421490,0.979,33.140,3866.780,-0.025,0.125287,3.310000
2003,0.622865,1.038,30.311,3115.573,0.004,0.060266,2.450000
2004,0.807549,1.027,33.690,3699.040,0.008,-0.010597,2.370000
2005,0.806305,0.988,37.058,4247.107,0.017,-0.037975,4.210000
2006,1.038258,1.009,44.525,5095.468,0.010,0.021255,2.910000
2007,0.505131,1.103,52.345,5752.749,0.006,0.093162,1.875000


## Estimation of the model

### we will use the Least Square Error Method to find the coefficients

In [256]:
print(dividend.head(5))
print(dividend.shift(-1).head(5))

year
1999    0.655
2000    0.752
2001    0.870
2002    0.979
2003    1.038
Name: Dividend, dtype: float64
year
1999    0.752
2000    0.870
2001    0.979
2002    1.038
2003    1.027
Name: Dividend, dtype: float64


In [293]:
dividend = df_yearly_final['Dividend']
dividend_rate = df_yearly_final['Dividend_Rate']
price = df_yearly_final['Price']
return_CAC40 = df_yearly_final['return_CAC40']
french_bonds_rate = df_yearly_final['french_bonds_rate']
beta = df_yearly_final['beta']

Y = (dividend.shift(-1)/price)-french_bonds_rate
A0 = beta*(return_CAC40 - french_bonds_rate)

Y = Y.values
A0 = A0.values
dividend_rate = dividend_rate.values

In [294]:
print(Y)
print(len(Y))
print(len(A0))
print(len(dividend_rate))

[-3.38297103 -0.67006812 -3.25013878 -3.27867833 -2.41611791 -2.34067379
 -4.18277241 -2.8852274  -1.85154026 -4.20804683 -3.61487185 -3.08963363
 -3.29447464 -2.50940048 -2.17808735 -1.64000135 -0.81885036 -0.43971176
 -0.78407289 -0.75900227 -0.10715026         nan]
22
22
22


In [295]:
# Get rid of the nan at the last value for all the inputs for the minimization
Y = Y[:-1]
A0 = A0[:-1]
dividend_rate = dividend_rate[:-1]

In [296]:
# function to minimize
def min_func(coef_A1):
    return Y-A0+coef_A1*dividend_rate

In [297]:
# initialization of the problem
N = 100
init_guess = np.ones(N)

In [298]:
min_func(init_guess)

array([12.0834162 , 14.24328108, 12.04108593, 10.65572745,  5.13396748,
       -1.49297257, -4.59940336,  2.25122831,  8.40870451,  9.95131982,
        9.02064284,  6.1650913 ,  5.94635479,  6.48830792,  6.68245534,
        6.68176943,  5.90126958,  5.13362587,  5.56051079,  5.44583375,
        4.7153581 ])

In [289]:
from scipy.optimize import minimize

opt_results = minimize(min_func,init_guess,method='SLSQP')

opt_results

ValueError: The user-provided objective function must return a scalar value.